In [1]:
import pandas as pd
import os
data_dirpath = '/usr2/mamille2/tumblr/data'

# See if can use data that already have for reblog prediction

## Load followers

In [2]:
follower_data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(follower_data))
print(follower_data.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

## Load followees

In [11]:
# Test follow structure

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, iterator=True)
follow_data = chunked.get_chunk(chunksize)
print(len(follow_data))
print(follow_data.columns)

100000
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [3]:
# See what follow structure can find for set of followers
from tqdm import tqdm_notebook as tqdm
import numpy as np

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

stx_follower_ids = set()
# follower_ids = follower_data['tumblog_id'].tolist()
# followee_data = []

for chunk in tqdm(chunked, total=total_iters):
    stx_follower_ids |= set(chunk['tumblog_id'].unique())
#     followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
    
print(len(stx_follower_ids))

follower_ids = stx_follower_ids.intersection(set(follower_data['tumblog_id']))
len(follower_ids) # Have at least one piece of follow information for these users


100861970


3044188

In [5]:
# Save out follower ids
with open(os.path.join(data_dirpath, 'halfday_followers_with_descriptions.txt'), 'w') as f:
    for i in sorted(follower_ids):
        f.write(f'{i}\n')

In [4]:
# Get followee data
follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

followee_data = [] # list of np arrays to be concatenated

for chunk in tqdm(chunked, total=total_iters):
    followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
follower_follow_data = pd.DataFrame(np.vstack(followee_data), columns=hdr)
len(follower_follow_data)

KeyboardInterrupt: 

In [29]:
# Save follow data from followers for which have descriptions
out_fname = 'follow_data_recent100.pkl'

follower_follow_data.to_pickle(os.path.join(data_dirpath, out_fname))

In [30]:
# For how many followees do we have blog descriptions for?
followees = set(follower_follow_data['followed_tumblog_id'])
print(len(followees))

followees_with_descriptions = followees.intersection()

6361471

## Load posts

In [ ]:
post_fname = 'textposts_recent100.pkl'
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))